# What am I Investigating?
Tennis is a racquet sport that can be played individually against a single opponent (singles) or between two teams of two players each (doubles). Players use a strung racquet to strike a hollow rubber ball, covered with felt, over a net and into the opponent's court. The objective of the game is to play the ball in such a way that the opponent cannot play a valid return. The player who is unable to return the ball will not gain a point, while the opposite player will.  
Tennis is played on different surfaces, including grass, clay, hard courts, and even indoor carpet. Each surface affects the ball's speed and bounce differently, leading to varied styles of play.  
*I'm intrested to find out if there's a relationship between the surface the match was played on and its duration*.

# Importing Libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

pd.set_option('display.max_columns', 50)

# Loading Data

In [2]:
folder_path = '/content/drive/MyDrive/Tennis_Analysis/full_matches_data'

# List all files in the directory with a .csv extension
all_files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f)) and f.endswith('.csv')]

# Use a list comprehension to read each file into a dataframe and then concatenate them all
combined_df = pd.concat([pd.read_csv(os.path.join(folder_path, f)) for f in all_files], ignore_index=True)

In [3]:
combined_df.sample(10)

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_seed,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
42914,2002-741,Nottingham,Grass,32,A,20020617,10,101885,NaN,NaN,Wayne Arthurs,L,190.0,AUS,31.2,102227,NaN,NaN,Sargis Sargsian,R,180.0,ARM,29.0,7-6(7) 6-7(4) 6-2,3,R32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,113.0,350.0,74.0,524.0
66294,2010-506,Buenos Aires,Clay,32,A,20100215,31,103507,2.0,NaN,Juan Carlos Ferrero,R,183.0,ESP,30.0,103970,1.0,NaN,David Ferrer,R,175.0,ESP,27.8,5-7 6-4 6-3,3,F,152.0,2.0,4.0,100.0,78.0,53.0,10.0,16.0,7.0,11.0,1.0,1.0,88.0,54.0,35.0,16.0,15.0,4.0,9.0,22.0,1760.0,19.0,1780.0
46523,2003-D069,Davis Cup G1 PO: VEN vs CHI,Hard,4,D,20030711,5,103065,NaN,NaN,Kepler Orellana,R,NaN,VEN,25.7,102701,NaN,NaN,Marcelo Rios,L,175.0,CHI,27.5,7-6(4) RET,5,RR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,475.0,47.0,44.0,860.0
39307,2001-560,US Open,Hard,128,G,20010827,46,102162,NaN,NaN,Chris Woodruff,R,188.0,USA,28.6,102179,NaN,NaN,Antony Dupuis,R,185.0,FRA,28.5,6-1 6-3 3-6 6-1,5,R128,133.0,4.0,7.0,109.0,69.0,51.0,20.0,16.0,9.0,11.0,8.0,7.0,94.0,57.0,41.0,12.0,16.0,5.0,12.0,90.0,465.0,60.0,661.0
50102,2005-314,Gstaad,Clay,32,A,20050704,4,103808,NaN,NaN,Potito Starace,R,188.0,ITA,23.9,102562,6.0,NaN,Jiri Novak,R,190.0,CZE,30.2,6-3 6-7(3) 7-5,3,R32,112.0,0.0,3.0,107.0,88.0,54.0,9.0,16.0,8.0,13.0,2.0,2.0,114.0,65.0,40.0,22.0,17.0,4.0,12.0,87.0,475.0,25.0,1215.0
94197,2019-0319,Kitzbuhel,Clay,32,A,20190729,293,106233,1.0,NaN,Dominic Thiem,R,185.0,AUT,25.9,124116,NaN,WC,Sebastian Ofner,R,191.0,AUT,23.2,6-3 6-2,3,R16,76.0,4.0,0.0,43.0,35.0,28.0,3.0,8.0,0.0,1.0,3.0,2.0,68.0,39.0,21.0,12.0,9.0,3.0,8.0,4.0,4595.0,167.0,307.0
22642,1996-330,Tel Aviv,Hard,32,A,19961014,5,101439,4.0,NaN,Javier Sanchez,R,178.0,ESP,28.7,102897,NaN,WC,Nir Welgreen,R,NaN,ISR,19.8,6-4 6-3,3,R32,68.0,5.0,5.0,67.0,37.0,29.0,15.0,10.0,5.0,6.0,1.0,5.0,59.0,42.0,27.0,9.0,9.0,1.0,4.0,46.0,863.0,229.0,164.0
70928,2011-439,Umag,Clay,28,A,20110725,3,104926,6.0,NaN,Fabio Fognini,R,178.0,ITA,24.1,104349,NaN,NaN,Rui Machado,R,178.0,POR,27.2,6-4 6-3,3,R32,87.0,3.0,1.0,55.0,28.0,18.0,12.0,9.0,1.0,4.0,0.0,5.0,74.0,48.0,24.0,12.0,10.0,5.0,11.0,38.0,1140.0,85.0,643.0
28564,1998-316,Bastad,Clay,32,A,19980706,23,103103,7.0,NaN,Dominik Hrbaty,R,183.0,SVK,20.5,103498,NaN,WC,Marat Safin,R,193.0,RUS,18.4,3-6 6-2 6-1,3,R16,81.0,10.0,7.0,82.0,44.0,34.0,16.0,12.0,4.0,6.0,5.0,1.0,83.0,38.0,24.0,20.0,12.0,7.0,12.0,44.0,879.0,73.0,661.0
17953,1995-379,Prague,Clay,32,A,19950731,28,101900,NaN,NaN,Nicklas Kulti,R,190.0,SWE,24.2,103151,NaN,Q,Mariano Zabaleta,R,183.0,ARG,17.4,6-2 6-1,3,QF,68.0,5.0,2.0,49.0,23.0,19.0,17.0,8.0,1.0,1.0,1.0,1.0,50.0,30.0,16.0,8.0,7.0,4.0,8.0,163.0,258.0,560.0,30.0


# Understanding the Big Picture

Getting the number of rows and columns in the dataset (pre-cleaning):

In [4]:
number_of_rows_pre_cleaning = combined_df.shape[0]
number_of_columns_pre_cleaning = combined_df.shape[1]

print(f"The dataset has {number_of_rows_pre_cleaning} rows and {number_of_columns_pre_cleaning} columns.")

The dataset has 104682 rows and 49 columns.


Counting the number of NA's in each column:

In [5]:
combined_df.isna().sum()

tourney_id                0
tourney_name              0
surface                   0
draw_size                 0
tourney_level             0
tourney_date              0
match_num                 0
winner_id                 0
winner_seed           62282
winner_entry          91873
winner_name               0
winner_hand               9
winner_ht              2454
winner_ioc                0
winner_age                5
loser_id                  0
loser_seed            81382
loser_entry           83599
loser_name                0
loser_hand               42
loser_ht               4855
loser_ioc                 0
loser_age                18
score                     0
best_of                   0
round                     0
minutes               13036
w_ace                 10207
w_df                  10207
w_svpt                10207
w_1stIn               10207
w_1stWon              10207
w_2ndWon              10207
w_SvGms               10206
w_bpSaved             10207
w_bpFaced           

Checking if there are any duplicate rows:

In [6]:
number_of_duplicated_rows = combined_df.duplicated().sum()
print(f"There are {number_of_duplicated_rows} duplicated rows in the dataset.")

There are 0 duplicated rows in the dataset.


# Data Cleaning

Deleting matches that we don't know their statistics:

In [7]:
# Creating a list of columns names that has the statistics
match_data_columns_names = ["w_ace", "w_df", "w_svpt", "w_1stIn", "w_1stWon", "w_2ndWon", "w_SvGms", "w_bpSaved", "w_bpFaced", "l_ace", "l_df", "l_svpt", "l_1stIn", "l_1stWon", "l_2ndWon", "l_SvGms", "l_bpSaved", "l_bpFaced"]

# Dropping all the rows based on the subset of columns in the list above
combined_df.dropna(subset=match_data_columns_names, inplace=True)

# Checking if it worked
combined_df[match_data_columns_names].isna().sum()

w_ace        0
w_df         0
w_svpt       0
w_1stIn      0
w_1stWon     0
w_2ndWon     0
w_SvGms      0
w_bpSaved    0
w_bpFaced    0
l_ace        0
l_df         0
l_svpt       0
l_1stIn      0
l_1stWon     0
l_2ndWon     0
l_SvGms      0
l_bpSaved    0
l_bpFaced    0
dtype: int64

Deleting unnecessary columns:

In [8]:
names_of_columns_to_drop = ["winner_seed", "winner_entry", "loser_seed", "loser_entry", "match_num", "match_num"]

combined_df.drop(columns=names_of_columns_to_drop, inplace=True)

combined_df.columns

Index(['tourney_id', 'tourney_name', 'surface', 'draw_size', 'tourney_level',
       'tourney_date', 'winner_id', 'winner_name', 'winner_hand', 'winner_ht',
       'winner_ioc', 'winner_age', 'loser_id', 'loser_name', 'loser_hand',
       'loser_ht', 'loser_ioc', 'loser_age', 'score', 'best_of', 'round',
       'minutes', 'w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon',
       'w_SvGms', 'w_bpSaved', 'w_bpFaced', 'l_ace', 'l_df', 'l_svpt',
       'l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved', 'l_bpFaced',
       'winner_rank', 'winner_rank_points', 'loser_rank', 'loser_rank_points'],
      dtype='object')

Getting the number of rows and columns in the dataset (post-cleaning):

In [9]:
number_of_rows_post_cleaning = combined_df.shape[0]
number_of_columns_post_cleaning = combined_df.shape[1]

print(f"The dataset has {number_of_rows_post_cleaning} rows and {number_of_columns_post_cleaning} columns.")
print(f"{number_of_rows_pre_cleaning-number_of_rows_post_cleaning} rows were deleted and {number_of_columns_pre_cleaning-number_of_columns_post_cleaning} were removed.")

The dataset has 94475 rows and 44 columns.
10207 rows were deleted and 5 were removed.


Changing the format of the dates in the tourney_date column, from `YYYYMMDD` format to `Y M D` format.

In [ ]:
combined_df['tourney_date'] = pd.to_datetime(combined_df['tourney_date'], format='%Y%m%d')
combined_df.head()

Creating new columns:

In [ ]:
# Creating a variable called "total_points" - the total points played in the match.
combined_df["total_points"] = combined_df["w_svpt"]+combined_df["l_svpt"]

combined_df.sample(7)

# Univariate Analysis

## Categorical Variables

Counting the number of games played on each surface type (and plotting it):

In [ ]:
combined_df.surface.value_counts()

Hard      47729
Clay      31085
Grass      9783
Carpet     5878
Name: surface, dtype: int64

In [ ]:
plt.figure(figsize=(12,6))
surfaces_countplot = sns.countplot(data=combined_df, x="surface", order=combined_df.surface.value_counts().index)
sns.despine(left=True, bottom=True)
surfaces_countplot.bar_label(surfaces_countplot.containers[0], label_type='edge')
surfaces_countplot.set_xlabel("Type of Surface")
surfaces_countplot.set_ylabel("Number of Matches Played")
surfaces_countplot.set_title("Number of Matches Played Between 1991 and 2023* on each surface")
plt.show()

Counting the number of times a best of 3 sets and a best of 5 sets were played (and plotting it):

In [ ]:
combined_df.best_of.value_counts()

In [ ]:
plt.figure(figsize=(12,6))
best_of_countplot = sns.countplot(data=combined_df, x="best_of", order=combined_df.best_of.value_counts().index)
sns.despine(left=True, bottom=True)
best_of_countplot.bar_label(surfaces_countplot.containers[0], label_type='edge')
best_of_countplot.set_xlabel("Best of How Many Sets")
best_of_countplot.set_ylabel("Number of Matches")
best_of_countplot.set_title("Number of Matches Played Between 1991 and 2023")
plt.show()

In [ ]:
combined_df.winner_hand.value_counts()

In [ ]:
combined_df.loser_hand.value_counts()

## Numerical Variables

Analyzing the total_points variable:

In [ ]:
combined_df.total_points.describe()

In [ ]:
plt.figure(figsize=(12, 6))
total_point_histogram = sns.histplot(data=combined_df, x="total_points")
sns.despine(left=True, bottom=True)
total_point_histogram.set_xlabel("Total Number of Points Played in a Match")
total_point_histogram.set_ylabel("Number of Matches")
plt.show()